In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
drive_path = '/content/drive'

# GPT-2 pré-treinada

GPT-2 pré-treinada pela OpenAI, disponibilizado por [Karpathy](https://github.com/karpathy/nanoGPT)

In [ ]:
!git clone https://github.com/karpathy/nanoGPT.git
!cp -r -n nanoGPT/* .

!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

!pip install tiktoken datasets

In [ ]:
# copying train and val files to folder data/
!mkdir data/messages
!mkdir data/telegram
!cp /content/drive/.../messages/train.bin data/messages/train.bin
!cp /content/drive/.../messages/val.bin data/messages/val.bin

In [3]:
import os, sys
import tempfile
import time
import pickle

import torch

from contextlib import nullcontext

import tiktoken

from model import GPT, GPTConfig

## Finetuning

In [4]:
out_dir = 'out'
dataset = 'telegram'
eval_interval = 5
eval_iters = 40
wandb_log = False # feel free to turn on
wandb_project = 'owt'
wandb_run_name = 'ft-' + str(time.time())

init_from = 'gpt2-medium' # gpt2 variant (e.g. 'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl')

# only save checkpoints if the validation loss improves
always_save_checkpoint = False

# the number of examples per iter:
# 1 batch_size * 32 grad_accum * 1024 tokens = 32,768 tokens/iter
# shakespeare has 301,966 tokens, so 1 epoch ~= 9.2 iters
batch_size = 1
gradient_accumulation_steps = 32
max_iters = 5000

# finetune at constant LR
learning_rate = 3e-5
decay_lr = False

In [5]:
# saving the parameters in a file in the Karpathy setup
config = {k: v for k, v in locals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))}
tmp = tempfile.NamedTemporaryFile(delete=False)
with open(tmp.name, 'w') as f:
    _ = [f.write('{} = "{}"\n'.format(k, v)) if isinstance(v, (str)) else f.write('{} = {}\n'.format(k, v)) for k, v in config.items()]

In [ ]:
# for large dataset, we prepared a script that makes all adjustments to the standard in batches
# download from https://drive.google.com/drive/folders/1JpPmDYhXgCGOFF1NNRj04fFqRkEIIYsQ?usp=sharing

# input_file_path = 'msgs-2023-1.csv'
# output_path = 'data/telegram'

# sys.argv = ['prepare', input_file_path, output_path]
# exec(open('data/telegram/prepare.py').read())

In [ ]:
sys.argv = ['finetuning', tmp.name]
exec(open('train.py').read())

In [ ]:
import shutil

zip_name = os.path.join(drive_path, 'out_dir_' + time.strftime("%Y%m%d-%H%M%S"))
shutil.make_archive(zip_name, 'zip', out_dir)

## Inference

In [4]:
import shutil

zip_name = os.path.join(drive_path, 'out_dir_20231201-123456.zip')
out_dir = 'out'

# unpacking the outdir from previous finetunnings
# make sure that the out_dir of the next cell will be the same as this one
shutil.unpack_archive(zip_name, out_dir, 'zip')

In [ ]:
init_from = 'resume' # reither 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl')
out_dir = 'out' # ignored if init_from is not 'resume'
num_samples = 10 # number of samples to draw
max_new_tokens = 280 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
device = 'cpu' # 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster

# saving the parameters in a temporary file in the Karpathy setup
config = {k: v for k,v in locals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))}
tmp = tempfile.NamedTemporaryFile(delete=False)
with open(tmp.name, 'w') as f:
    _ = [f.write('{} = "{}"\n'.format(k, v)) if isinstance(v, (str)) else '{} = {}\n'.format(k, v) for k, v in config.items()]

sys.argv = ['inference', tmp.name]
exec(open('configurator.py').read()) # overrides from command line or config file

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

In [ ]:
if init_from == 'resume':
    # init from a model saved in a specific directory
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    gptconf = GPTConfig(**checkpoint['model_args'])
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
elif init_from.startswith('gpt2'):
    # init from a given GPT-2 model
    model = GPT.from_pretrained(init_from, dict(dropout=0.0))

In [ ]:
pikle_model_name = os.path.join(drive_path, time.strftime("%Y%m%d-%H%M%S") + '.pickle')

with open(pikle_model_name, "wb") as fp:
    pickle.dump({
        'model': model,
        'device': device,
        'ctx': ctx,
        'num_samples': num_samples,
        'max_new_tokens': max_new_tokens,
        'temperature': temperature,
        'top_k': top_k
    }, fp, pickle.HIGHEST_PROTOCOL)

print(pikle_model_name)

In [7]:
# a sentence or etc. Can also specify a file, use as: "FILE:prompt.txt"

start = "Vacinas adicionam um chip no corpo"

In [8]:
# gpt-2 encodings
enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode_ordinary(s)
decode = lambda l: enc.decode(l)

# encode the beginning of the prompt
if start.startswith('FILE:'):
    with open(start[5:], 'r', encoding='utf-8') as f:
        start = f.read()
start_ids = encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

In [ ]:
# run generation
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            print(decode(y[0].tolist()))
            print('---------------')